# 4th lab:

### Area info:

In [249]:
bottom_left = (29.073321247506765, 49.845775018245774)
upper_right = (31.986007792928522, 51.278667808079206)

#Filtering options:
from_time = "2019-04-01T00:00:00Z"
to_time = "2020-05-01T00:00:00Z"
maxCloudCoverage = 0

#In px:
width = 2000
height = 2000

## First part:

### Create session with server:

In [330]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

def get_token():
    client_id = "8d7432b3-19c9-4a6e-9d02-fcf865d1a6c6"
    client_secret = "fdR{0FFu(}F)KD/s@>9(w;28*]MHkQJoo_I!*Q73"


    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client)


    token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                              client_id=client_id, client_secret=client_secret)
    return token['access_token']

token = get_token()

### get_band(band_num: int, img_format: str ="tiff"):
___

Takes as arguments band number and image format to save. It downloads and saves 1-band image with target band to *./data/{band name}.{img_format}*
___

In [260]:
import requests
import os

def get_band(band_num: int, img_format: str = "tiff"):
    band_names = {
        2: "b",
        3: "g",
        4: "r",
        8: "nir",
    }
    req_data = {
        "input": {
            "bounds": {
                "bbox": [
                    bottom_left[0],
                    bottom_left[1],
                    upper_right[0],
                    upper_right[1],
                ]
            },
            "data": [
                {
                    "type": "S2L2A",
                    "dataFilter": {
                        "timeRange": {
                            "from": from_time,
                            "to": to_time,
                        },
                        "maxCloudCoverage": maxCloudCoverage,
                    },
                }
            ]
        },
        "output": {
            "width": width,
            "height": height,
            "responses": [{
                "format": {
                    "type": "image/" + img_format
                }
            }]
        },
        "evalscript": """
        //VERSION=3

        function setup() {
          return {
            input: ["B0""" + str(band_num) + """"], 
            output: {
              bands: 1
            }
          };
        }

        function evaluatePixel(
          sample,
          scenes,
          inputMetadata,
          customData,
          outputMetadata
        ) {
          return [2.5 * sample.B0""" + str(band_num) + """]; 
        }
        """
    }
    # 0 * sample.B04, 0 * sample.B03, 2.5 * sample.B02, 4-3-2-8 #"B03", "B04", "B08"
    headers = {
        "Authorization": f"Bearer {token['access_token']}",
    }
    url = "https://services.sentinel-hub.com/api/v1/process"

    response = requests.post(url, json=req_data, headers=headers)
    
    if not os.path.isdir("data"):
        os.mkdir("data")
    
    with open(f"data/{band_names[band_num]}.{img_format}", "wb") as img:
        img.write(response.content)

### Getting all necessary bands:

In [261]:
get_band(2)
get_band(3)
get_band(4)
get_band(8)

In [347]:
def get_image(satellite: str, bands, merge=True, img_name: str = "image", img_format: str = "tiff"):
    if not os.path.isdir("data"):
        os.mkdir("data")  
    if not os.path.isdir(f"data/{satellite}"):
        os.mkdir(f"data/{satellite}")
        
    if type(bands) is not type(list()):
        bands = [bands]
    
    
    band_names = {
        4: "r", 
        3: "g", 
        2: "b", 
        8: "b08",
    }
    band_id = {
        4:"B04",
        3:"B03",
        2:"B02",
        8:"B08",
    }
    satellite_id = {
        "sentinel": "S2L2A",
        "landsat": "L8L1C",
    }
    urls = {
        "sentinel": "https://services.sentinel-hub.com/api/v1/process",
        "landsat": "https://services-uswest2.sentinel-hub.com/api/v1/process",
    }
    
    
    if not merge:
        for band in bands:
            get_image(satellite, band, img_format=img_format, merge=True, img_name=band_names[band])
        return

    
    evaluate_pixel_output = "["
    for i in list(band_id.keys()):
        if i in bands:
            evaluate_pixel_output += "2.5 * sample." + band_id[i] + ", "
    evaluate_pixel_output = evaluate_pixel_output[:-2] + "]"
    
    
    evalscript = """
        //VERSION=3

        function setup() {
          return {
            input: """ + str([band_id[i] for i in bands]) + """, 
            output: {
              bands: """ + str(len(bands)) + """
            }
          };
        }

        function evaluatePixel(
          sample,
          scenes,
          inputMetadata,
          customData,
          outputMetadata
        ) {
          return """ + evaluate_pixel_output + """; 
        }
        """
    
    # Checking token:
    try:
        resp = oauth.get("https://services.sentinel-hub.com/oauth/tokeninfo")
    except Exception as e:
        if e.error == "token_expired":
            token = get_token()    
        
    req_data = {
        "input": {
            "bounds": {
                "bbox": [
                    bottom_left[0],
                    bottom_left[1],
                    upper_right[0],
                    upper_right[1],
                ]
            },
            "data": [
                {
                    "type": satellite_id[satellite],
                    "dataFilter": {
                        "timeRange": {
                            "from": from_time,
                            "to": to_time,
                        },
                        "maxCloudCoverage": maxCloudCoverage,
                    },
                }
            ]
        },
        "output": {
            "width": width,
            "height": height,
            "responses": [{
                "format": {
                    "type": "image/" + img_format
                }
            }]
        },
        "evalscript": evalscript
    }
    headers = {
        "Authorization": f"Bearer {token}",
    }
    url = urls[satellite]
            

    response = requests.post(url, json=req_data, headers=headers)

    with open(f"data/{satellite}/{img_name}.{img_format}", "wb") as img:
        img.write(response.content)

In [348]:
get_image("landsat", bands=[2, 3, 4])

# 